In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.optimize import curve_fit
from sklearn.neighbors import KernelDensity
from sklearn.linear_model import LinearRegression
from scipy.signal import bessel, sosfiltfilt
from sklearn.cluster import KMeans

from sklearn.neighbors import KernelDensity
from data_manager import DataManager
import transport_signal_processing as tsp

In [2]:
# parameters
path = "*"
re_sel = "AA00300AA"
level = 1
selected_only = True

# setup database connector
sigman = DataManager('database')

# load segments informations
sinfo_l = sigman.load_info(path, 's*')

# convert info to dataframe
df = pd.DataFrame(sinfo_l)

# keep only selected polymers in dataframe and signal info
df = df[df['analyte'].str.match(re_sel) & (df['selected'] > (level-1))]
sinfo_l = [sinfo_l[i] for i in df.index.values]
df = df.reset_index(drop=True)
   
# load events core
cores = tsp.utils.load_core_events(sigman, sinfo_l, selected_only=True)

# debug print
display(df)
print("{} events".format(len(cores)))

,pore,temperature,voltage,analyte,buffer,channel,id,sid,segment_range,segment_duration,MODIFIED,mI_open,sI_open,N_events,N_cores,N_reduced,selected,ratio_sel
0,K238A,25,100,AA00300AA,LiCl,4,1-0,0,"[0, 4246287]",42.46287,2022-03-10_09:58:55,44.726295,3.441116,2176.0,1001.0,1001.0,1.0,0.825175
1,K238A,25,100,AA00300AA,LiCl,4,2-0,0,"[0, 5121195]",51.21195,2022-03-10_09:58:43,43.902667,3.574460,2219.0,1075.0,1075.0,1.0,0.817674
2,K238A,25,100,AA00300AA,LiCl,4,2-0,1,"[5185984, 8182573]",29.96589,2022-03-10_09:58:43,44.055105,3.615654,1333.0,647.0,647.0,1.0,0.799073
3,K238A,25,100,AA00300AA,LiCl,4,2-0,2,"[8434663, 15932687]",74.98024,2022-03-10_09:58:55,43.880660,3.592745,3801.0,1799.0,1799.0,1.0,0.831017
4,K238A,25,100,AA00300AA,LiCl,2,3-0,4,"[25490928, 37941599]",124.50671,2022-03-10_09:58:43,41.274651,3.066304,4435.0,2356.0,2356.0,1.0,0.938455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,K238A,15,100,AA00300AA,KCl,4,6,0,"[0, 18476959]",184.76959,2022-03-10_09:58:39,67.498344,3.551380,3543.0,1183.0,1183.0,1.0,0.770076
253,K238A,15,100,AA00300AA,KCl,4,6,1,"[18478922, 35137084]",166.58162,2022-03-10_09:58:39,67.301492,3.524443,2308.0,855.0,855.0,1.0,0.767251
254,K238A,15,100,AA00300AA,KCl,4,6,2,"[35187376, 113683647]",784.96271,2022-03-10_09:58:53,67.802013,3.509333,14926.0,5144.0,5144.0,1.0,0.769440
255,K238A,5,100,AA00300AA,KCl,4,6-0,0,"[0, 90978255]",909.78255,2022-03-10_09:58:54,53.412203,3.143489,12633.0,3148.0,3148.0,1.0,0.563532


335537 events


In [3]:
def envelop_filtering(I, ws=5, n_iters=256):
    # envelope iterative smoothing
    Is = I.copy()
    for k in range(n_iters):
        Imax, Imin = tsp.signals.envelope(Is, ws)
        Is = 0.5*(Imax + Imin)

    # steps location
    ids_step = np.where(np.abs(np.diff(np.sign(np.diff(Is)))) > 0.5)[0]

    # steps amplitudes
    dI = np.abs(np.diff(Is[ids_step]))
    
    return Is, ids_step, dI

In [4]:
# parameters
N_interp = 200
smoothing = True
ws = 1
n_iters = 64

# interpolate events
if smoothing:
    X = np.array([tsp.signals.downsample(envelop_filtering(x[:,1], ws=ws, n_iters=n_iters)[0], N_interp) for x in tqdm(cores)])
else:
    X = np.array([tsp.signals.downsample(x[:,1], N_interp) for x in tqdm(cores)])

 26%|██▋       | 88150/335537 [10:03<29:20, 140.51it/s]

KeyboardInterrupt: 

In [ ]:
# parameters
N_clst = 64

# clustering
clst = KMeans(N_clst)
clst.fit(X)

# get labels
y = clst.labels_

# split by clusters
X_l = []
ic_l = []
for i in np.unique(y):
    ids = np.where(y == i)[0]
    X_l.append(X[ids])
    ic_l.append(ids)

# get cluster centers
Xc = clst.cluster_centers_

In [ ]:
def gauss(x, A, mu, sigma):
    return np.abs(A) * np.exp(-np.square((x - mu))/np.square(sigma))


def multi_gauss(x, *p):
    y = np.zeros(x.shape)
    for k in range(int(len(p) / 3)):
        y += gauss(x, p[3*k], p[3*k+1], p[3*k+2])

    return y


def multi_gauss_dist_fit(x, y, num_gauss=1):
    # create guess for fit
    guess = np.array([[np.max(y), np.linspace(np.min(x), np.max(x), num_gauss+2)[1:-1][k], np.std(x)] for k in range(num_gauss)]).ravel()

    # perform multi gauss fit
    try:
        popt, pcov = curve_fit(multi_gauss, x.astype(np.float64), y.astype(np.float64), p0=guess.astype(np.float64))
    except RuntimeError:
        print("ERROR: gaussian fit did not converge")
        popt = np.zeros(guess.shape)
    except Exception as e:
        print("ERROR: {}".format(e))
        popt = np.zeros(guess.shape)

    # compute gaussian functions
    z = multi_gauss(x, *popt)

    return z, popt

In [ ]:
def levels_probabilites(x, popt_l):
    z_l = []
    for i in range(len(popt_l)):
        popt = popt_l[i]
        z_l.append(tsp.fits.gauss(x, popt[0], popt[1], popt[2]))
    
    return np.stack(z_l, axis=1)

In [ ]:
# parameters
I_lims = (10.0, 60.0)
t_lims = (1e-1, 1e2)
threshold = 0.2
ws = 1
n_iters = 64

# define x axis points
t = np.linspace(0.0, 1.0, N_interp)

# sort by cluster size
M = sum([len(x) for x in X_l])
ids_srtd = np.argsort([1e2*x.shape[0]/M for x in X_l])[::-1]

for k in ids_srtd:
    plt.figure(figsize=(16,3))

    # get stats
    cores_clust = [cores[i] for i in ic_l[k]]
    mI = np.array([np.mean(c[:,1]) for c in cores_clust])
    dwt = np.array([c[-1,0]-c[0,0] for c in cores_clust])

    # subplot
    plt.subplot(131)
    plt.semilogy(mI, dwt, 'k.', ms=1.0, alpha=0.5)
    plt.xlim(0.0, 1.0)
    plt.xlim(I_lims)
    plt.ylim(t_lims)
    plt.title('<I>={:.1f}±{:.1f}, <dwt>={:.1f}±{:.1f}ms'.format(np.mean(mI), np.std(mI), np.mean(dwt), np.std(dwt)))
    
    # fits
    #Idist = np.concatenate([c[:,1] for c in cores_clust])
    #Idist = np.concatenate([c[np.where(np.abs(np.diff(np.sign(np.diff(c[:,1])))) > 0.5)[0],1] for c in cores_clust])
    Idist = []
    for c in cores_clust:
        Is, ids_step, _ = envelop_filtering(c[:,1], ws=ws, n_iters=n_iters)
        Idist.append(Is[ids_step])
    Idist = np.concatenate(Idist)
    
    # KDE
    kde = KernelDensity(bandwidth=0.5)
    kde.fit(Idist.reshape(-1,1))
    x = np.linspace(I_lims[0], I_lims[1], 100)
    y = np.exp(kde.score_samples(x.reshape(-1,1))).ravel()

    # compute integral
    s0 = np.sum(y)

    # perform iterative gaussian fits until only "thr" of the distribution is unexplained
    for i in range(3):
        z, popt = multi_gauss_dist_fit(x, y, num_gauss=i+1)

        # compute residual error
        s = np.sum(np.abs(y - z))
        r = s/s0

        # pack results
        popt_l = [popt[3*i:3*(i+1)] for i in range(i+1)]

        # evaluate result
        if r < threshold:
            break
    
    # subplot
    plt.subplot(132)
    plt.hist(Idist, density=True, range=I_lims, bins=80)
    plt.plot(x,y,'k-')
    for i in range(len(popt_l)):
        popt = popt_l[i]
        z = tsp.fits.gauss(x, popt[0], popt[1], popt[2])
        plt.plot(x,z,label="μ={:.2f}, σ={:.2f}".format(popt[1], popt[2]))
    plt.xlabel('relative current')
    plt.ylabel('density')
    plt.legend(loc='lower center')
    plt.xlim(I_lims)
    
    # find levels
    a_lvls = np.argmax(levels_probabilites(Xc[k], popt_l), axis=1)
    ids_lvls = [np.where(a_lvls==i)[0] for i in range(len(popt_l))]
    
    # subplot
    plt.subplot(133)
    for x in X_l[k]:
        plt.plot(t, x, 'k-', alpha=0.025)
    for ids_lvl in ids_lvls:
        plt.plot(t[ids_lvl], Xc[k][ids_lvl], 'd')
    plt.plot(t, Xc[k], 'r-')
    plt.xlim(0.0, 1.0)
    plt.ylim(I_lims)
    plt.xticks([0.0, 0.25, 0.5, 0.75, 1.0], ['', '', '', '', ''])
    plt.title('[{}] {:.2f}% ({})'.format(k, 1e2*X_l[k].shape[0]/M, X_l[k].shape[0]))

    plt.tight_layout()
    plt.show()
    
    if k == 49:
        break

In [ ]:
ids_opts = np.where(np.abs(np.diff(np.sign(np.diff(Xc[k])))) > 0.5)[0]
ids_opts

In [ ]:
plt.figure()
plt.plot(t, Xc[k])
plt.plot(t[ids_opts+1], Xc[k][ids_opts+1], '.')
plt.ylim(25.0, 45.0)
plt.show()

In [ ]:
def dtw(s, t):
    n, m = len(s), len(t)
    dtw_matrix = np.zeros((n+1, m+1))
    for i in range(n+1):
        for j in range(m+1):
            dtw_matrix[i, j] = np.inf
    dtw_matrix[0, 0] = 0
    
    for i in range(1, n+1):
        for j in range(1, m+1):
            cost = abs(s[i-1] - t[j-1])
            # take last min from a square box
            last_min = np.min([dtw_matrix[i-1, j], dtw_matrix[i, j-1], dtw_matrix[i-1, j-1]])
            dtw_matrix[i, j] = cost + last_min
    return dtw_matrix

In [ ]:
dtw(cores[0][:,1], cores[1][:,1])

In [ ]:
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

In [ ]:
def envelop_features(I, ws=5, n_iters=256):
    # envelope iterative smoothing
    Is = I.copy()
    for k in range(n_iters):
        Imax, Imin = tsp.signals.envelope(Is, ws)
        Is = 0.5*(Imax + Imin)

    # steps location
    ids_step = np.where(np.abs(np.diff(np.sign(np.diff(Is)))) > 0.5)[0]

    # steps amplitudes
    dI = np.abs(np.diff(Is[ids_step]))
    
    return Is[ids_step], ids_step

In [ ]:
i = np.random.choice(len(cores))
j = np.random.choice(len(cores))
#i,j = (7214, 6100)

ti, Ii = cores[i].T
tj, Ij = cores[j].T
ti = np.linspace(0.0, 1.0, Ii.shape[0])
tj = np.linspace(0.0, 1.0, Ij.shape[0])

xi, ids_i = envelop_features(Ii, ws=1, n_iters=8)
xj, ids_j = envelop_features(Ij, ws=1, n_iters=8)

distance, path = fastdtw(xi, xj, dist=euclidean)
print(distance)

s = np.max(Ii)

plt.figure()
plt.plot(ti, Ii)
plt.plot(tj, Ij+s)
for pi, pj in path:
    ki = ids_i[pi]
    kj = ids_j[pj]
    plt.plot([ti[ki], tj[kj]], [Ii[ki], Ij[kj]+s], '-', color='gray')
plt.plot(ti[ids_i], Ii[ids_i], 'd')
plt.plot(tj[ids_j], Ij[ids_j]+s, 'd')
plt.show()

In [ ]:
N = len(cores)
i = np.random.choice(len(cores))
ti, Ii = cores[i].T
ti = np.linspace(0.0, 1.0, Ii.shape[0])
xi, ids_i = envelop_features(Ii, ws=1, n_iters=8)

plt.figure()
plt.plot(ti, Ii)
plt.plot(ti[ids_i], Ii[ids_i], 'd')
plt.show()

In [ ]:
D = np.zeros((N,))
paths = []
for j in tqdm(range(N)):
    tj, Ij = cores[j].T
    tj = np.linspace(0.0, 1.0, Ij.shape[0])

    xj, ids_j = envelop_features(Ij, ws=1, n_iters=8)

    distance, path = fastdtw(xi, xj, dist=euclidean)

    D[j] = distance
    paths.append(path)

In [ ]:
for j in np.argsort(D)[1:][:3]:
    tj, Ij = cores[j].T
    tj = np.linspace(0.0, 1.0, Ij.shape[0])

    xj, ids_j = envelop_features(Ij, ws=1, n_iters=8)

    distance, path = fastdtw(xi, xj, dist=euclidean)

    s = np.max(Ii)

    plt.figure(figsize=(16,4))
    plt.plot(ti, Ii)
    plt.plot(tj, Ij+s)
    for pi, pj in path:
        ki = ids_i[pi]
        kj = ids_j[pj]
        plt.plot([ti[ki], tj[kj]], [Ii[ki], Ij[kj]+s], '-', color='gray')
    plt.plot(ti[ids_i], Ii[ids_i], 'd')
    plt.plot(tj[ids_j], Ij[ids_j]+s, 'd')
    plt.title("{:.3f}".format(distance))
    plt.show()

In [ ]:
plt.figure()
plt.hist(D, bins=100, range=(0.0, 1000.0))
plt.show()

np.sum(D < 200)

In [ ]:
# parameters
N = 1000

# aggregated core
agg_core = []
for j in tqdm(np.argsort(D)[1:][:N]):
    tj, Ij = cores[j].T
    tj = np.linspace(0.0, 1.0, Ij.shape[0])

    xj, ids_j = envelop_features(Ij, ws=1, n_iters=8)

    distance, path = fastdtw(xi, xj, dist=euclidean)

    ac = [[] for _ in range(xi.shape[0])]
    for pi,pj in path:
        ac[pi].append(xj[pj])
        
    acc = []
    for i in range(xi.shape[0]):
        acc.append(ac[i][np.argmin(np.abs(ac[i]-xi[i]))])

    agg_core.append(acc)
    
agg_core = np.array(agg_core).T

In [ ]:
# sigma
s = np.array([np.std(ac) for ac in agg_core])

# plot
plt.figure(figsize=(16,4))
plt.violinplot([a for a in agg_core], ti[ids_i], widths=0.02)
plt.plot(ti, Ii)
#plt.errorbar(ti[ids_i], Ii[ids_i], yerr=s, fmt='d', color='crimson')
plt.fill_between(ti[ids_i], Ii[ids_i]-s, Ii[ids_i]+s, alpha=0.2, color='crimson')
plt.xlim(ti[0], ti[-1])
plt.show()

In [ ]:
plt.figure(figsize=(16,4))
plt.plot(ti, Ii, 'k-', alpha=0.2)
plt.scatter(ti[ids_i], Ii[ids_i], c=s, marker='s', cmap='turbo_r', vmin=2.0, vmax=10.0)
plt.colorbar()
plt.xlim(ti[0], ti[-1])
plt.show()

In [ ]:
# parameters
s_thr = 4.0
m = (s < s_thr)

# plot
plt.figure(figsize=(16,4))
plt.plot(ti, Ii, 'k-', alpha=0.2)
plt.scatter(ti[ids_i][m], Ii[ids_i][m], c=s[m], marker='s', cmap='turbo_r', vmin=2.0, vmax=10.0)
plt.colorbar()
plt.xlim(ti[0], ti[-1])
plt.show()

In [ ]:
dI = np.array([min(np.abs(Ii[ids_i[i]] - Ii[ids_i[i+1]]), np.abs(Ii[ids_i[i]] - Ii[ids_i[i-1]])) for i in range(1,len(ids_i)-1)])

# plot
plt.figure()
plt.plot(dI, s[1:-1], '.')
plt.xlabel('ΔI')
plt.ylabel('σ')
plt.show()

In [ ]:
x = [np.array([1,2,3]), np.array([4,5])]

In [ ]:
np.save("test.npy", x, allow_pickle=True)

In [ ]:
y = np.load("test.npy", allow_pickle=True)

In [ ]:
y[0].dtype

In [ ]:
# parameters
n_ref = 100
n_sample = 200
n_sel = 20

# statistics on aggregated dynamic time warping
dI, s = [], []
for i in tqdm(np.random.choice(len(cores), n_ref, replace=False)):
    # get reference event
    ti, Ii = cores[i].T
    ti = np.linspace(0.0, 1.0, Ii.shape[0])
    xi, ids_i = envelop_features(Ii, ws=1, n_iters=8)

    # compute features, distances and paths
    distances = []
    paths = []
    xj_l = []
    for j in np.random.choice(len(cores), n_sample, replace=False):
        if i == j:
            continue

        tj, Ij = cores[j].T
        tj = np.linspace(0.0, 1.0, Ij.shape[0])

        xj, _ = envelop_features(Ij, ws=1, n_iters=8)

        distance, path = fastdtw(xi, xj, dist=euclidean)

        distances.append(distance)
        paths.append(path)
        xj_l.append(xj)

    # aggregated core
    agg_core = [[] for _ in range(xi.shape[0])]
    for j in np.argsort(distances)[:n_sel]:
        xj = xj_l[j]
        path = paths[j]
        for pi,pj in path:
            agg_core[pi].append(xj[pj])
            
    # aggregated core
    agg_core = []
    for j in np.argsort(distances)[:n_sel]:
        xj = xj_l[j]
        path = paths[j]

        ac = [[] for _ in range(xi.shape[0])]
        for pi,pj in path:
            ac[pi].append(xj[pj])

        acc = []
        for i in range(xi.shape[0]):
            acc.append(ac[i][np.argmin(np.abs(ac[i]-xi[i]))])

        agg_core.append(acc)

    agg_core = np.array(agg_core).T

    s.append(np.array([np.std(ac) for ac in agg_core])[1:-1])
    dI.append(np.array([min(np.abs(Ii[ids_i[i]] - Ii[ids_i[i+1]]), np.abs(Ii[ids_i[i]] - Ii[ids_i[i-1]])) for i in range(1,len(ids_i)-1)]))
    
# pack data
s = np.concatenate(s)
dI = np.concatenate(dI)

In [ ]:
# fit
a, b = np.polyfit(dI, s, 1)
x = np.linspace(np.min(dI), np.max(dI))
y = a*x+b

# plot
plt.figure()
plt.plot(dI, s, '.')
plt.plot(x, y, 'k--', label='{:.2f}x+{:.2f}'.format(a,b))
plt.ylim(0.0, 12.0)
plt.legend(loc='best')
plt.xlabel('ΔI')
plt.ylabel('σ')
plt.show()

* y = 0.13x + 4.01
* y = 0.13x + 4.25

In [ ]:
plt.figure()
plt.hist(s, bins=40, range=(0.0, 10.0))
plt.show()

np.quantile(s, 0.2)